# Reinforcement Learning: Frozen Lake Problem

In this notebook we will learn about reinforcement learning using [Open AI Gym's Frozen Lake environment](https://gym.openai.com/envs/FrozenLake-v0/). Read over the description in the link so that you understand the environment.

Specifically, we will use a technique called **Value Iteration** to solve the Markov Decision Process created by Open AI Gym. With this technique we build up our agent's policy by repeatedly looping through all possible states and determining how "good" any given state is, or how much *value* it adds towards reaching our goal.

We start by importing the necessary modules and creating the environment.

In [16]:
# !pip install --upgrade gym==0.17.3
import gym
import time
import statistics

env = gym.make('FrozenLake8x8-v0')

Here we will define a function to determine the value or "goodness" of any given action.

The function works as follows:

We loop through all possible states that could follow from the given action, these possibile states are given by `possibilities`. The environment gives us the probability of going to that state as well as the reward we receive for moving to that state.

The formula for the goodness of an action from the current state takes into account the reward for moving to that state, as well how "good" we had previously determined this state was, then it factors in the probability of actually moving to this particular state. It does this for all possible states and sums them up to figure out how "good" this action is.

The `gamma` variable here determines how much our previous determinations
of the next state's goodness factors into our decision.

`env.P[state][action]` here serves to provide us the values of $P_a(s, s')$ and $R_a(s, s')$ from the MDP for every possible next state $s'$ from our current state (`state`) given `action`. Specifically, it gives a sequence of tuples containing the probability ($P_a(s, s')$), the next state ($s'$), and the reward ($R_a(s, s')$).

This function corresponds to the

$$
\sum_{s'} P_a(s', s)(R_a(s,s') + \gamma V_i(s'))
$$

portion of value iteration formula.

In [2]:
# Note: this can be added in later once it is needed, might make more sense then
def calc_action_value(state: int, action: int, state_func: list,
                      gamma: float) -> float:
    action_value = 0
    # env.P[state][action] gives us the possible next states from
    # `state` by taking `action`
    for prob, next_state, reward, _ in env.P[state][action]:
        action_value += prob * (reward + gamma * state_func[next_state])

    return action_value

Next we implement the Value Iteration algorithm. The purpose of this algorithm is to assign a value to each state. The greater the value of a state, the better that state is for accomplishing our goal. We accomplish this by repeatedly looping through all the states in our environment, then computing the value of that state based on the rewards available to us from that state. 

On repeated iterations through the states, the previously computed value of each state is also factored into a state's value calculation. For example, if we determine that one state has a high value because it is right next to the goal, then these repeated iterations would determine that states around this state also have a high value because the goal can be reached from them.

Here is the algorithm for value iteration again:

Loop until the left hand side is approximately equal to the right hand side:

$$
V_{i+1}(s) := \max_a \bigg\{ \sum_{s'} P_a(s', s)(R_a(s,s') + \gamma V_i(s')) \bigg\}
$$

where $i$ is the iteration number, and $s$ is the current state which we are
calculating the goodness of.


In our code, `state_func` is the value of $V_i$, `new_state_func` is the value of $V_{i+1}$, `state` is $s$, `action` is $a$, `i` is $i$, and `gamma` is $\gamma$.

In [22]:
# def value_iteration(env, max_iterations=100000, gamma=0.9):
#     """Determines how "good" any given state is to be in for our actor.

#     Returns: an array with a value for each state, and another array for the policy.
#         The greater a given value in the first array, the "better" the state.
#     """
#     # An array where each item represents how "good" a state is to be in
#     state_func = [0] * env.nS
#     # We'll update the state function gradually and use this copy to do it
#     new_state_func = state_func.copy()
#     # Our policy will contain the best action for any given state
#     # Do this after the initial value iteration
#     policy = [0] * env.nS

#     # Prevent looping infinitely if our algorithm doesn't converge
#     for i in range(max_iterations):
#         # Loop through all possible states
#         for state in range(env.nS):
#             # For each state we find the best possible action to take in that state
#             best_state_action_val = 0
#             # Do this after the initial value iteration
#             best_action = 0
#             # So we try all the actions
#             for action in range(env.nA):
#                 state_action_val = calc_action_value(state, action, state_func, gamma)

#                 # After calculating the goodness of this action, we keep it only if it is
#                 # better than the previous best
#                 if state_action_val > best_state_action_val:
#                     best_state_action_val = state_action_val
#                     # Do this after the initial value iteration
#                     best_action = action

#             # After calculating the best possible action for this state, we save how
#             # "good" the best action is for the state...
#             new_state_func[state] = best_state_action_val
#             # And we remember the action for later
#             # Do this after the initial value iteration
#             policy[state] = best_action

#         # After 1000 iterations, if the state function hasn't improved very much
#         # we stop trying to improve it
#         if i > 1000 and sum(state_func) - sum(new_state_func) < 1e-04:
#             break

#         # Otherwise we update the state function to the newly improved version
#         state_func = new_state_func.copy()

#     # After figuring out the goodness of each state and the best actions we return them
#     return state_func, policy

def value_iteration(env, max_iterations=100000, gamma=0.9):
    """Determines how "good" any given state is to be in for our actor.

    Returns: an array with a value for each state, and another array for the policy.
        The greater a given value in the first array, the "better" the state.
    """
    # An array where each item represents how "good" a state is to be in
    state_func = [0] * env.nS
    # We'll update the state function gradually and use this copy to do it
    new_state_func = state_func.copy()
    # Our policy will contain the best action for any given state
    # Do this after the initial value iteration
    # best_actions = [0] * env.nS

    # Prevent looping infinitely if our algorithm doesn't converge
    for i in range(max_iterations):
        # Loop through all possible states
        for state in range(env.nS):
            # For each state we find the best possible action to take in that state
            best_state_action_val = 0
            # Do this after the initial value iteration
            best_action = 0
            # So we try all the actions
            for action in range(env.nA):
                state_action_val = calc_action_value(state, action, state_func, gamma)

                # After calculating the goodness of this action, we keep it only if it is
                # better than the previous best
                if state_action_val > best_state_action_val:
                    best_state_action_val = state_action_val
                    # Do this after the initial value iteration
                    best_action = action
                    # best_actions[state] = action
                    
            # After calculating the best possible action for this state, we save how
            # "good" the best action is for the state...
            new_state_func[state] = best_state_action_val
            # And we remember the action for later

        # After 1000 iterations, if the state function hasn't improved very much
        # we stop trying to improve it
        if i > 1000 and sum(state_func) - sum(new_state_func) < 1e-04:
            break

        # Otherwise we update the state function to the newly improved version
        state_func = new_state_func.copy()
    return state_func

def get_policy(env, state_func, gamma=0.9):
    policy = [0] * env.nS
    current_state = 0
    # Loop through all possible states
    for state in range(env.nS):
        for action in range(env.nA):
            if state_func[state] == calc_action_value(state, action, state_func, gamma):
                policy[state] = action
    # After figuring out the goodness of each state and the best actions we return them
    return policy

env.reset()
state_func = value_iteration(env)
poli = get_policy(env, state_func)
print(poli)

[3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 1, 3, 3, 0, 3, 2, 3, 2, 1, 3, 3, 3, 3, 0, 3, 2, 1, 3, 3, 3, 3, 2, 1, 3, 2, 0, 3, 3, 2, 3, 0, 3, 2, 0, 3, 2, 3, 3, 2, 3, 2, 0, 1, 0, 3, 2, 1, 1, 3]


Now that we have the value or "goodness" of any given state, we need to use these values to build a *policy*. The policy defines the action that should be taken from any given state. This action should be based on the value of the states around the given state.

For example, if we are in state `A` and moving right means we are likely to move to state `B` which a value of 2, and moving left means moving to state `C` which has a value of 1, our policy should tell us that we should take the "right" action instead of the "left".

Let's go through our value iteration function above now and update it to remember the best action for each state.

Now our agent is ready to navigate its way through the frozen lake! It has a policy which will tell it the best action to take in any of the 64 states, so all we have to do is tell it to act based on the policy.

In [6]:
def get_score(env, policy, episodes=1000):
    misses = 0
    steps_list = []
    best_episode = []
    # We try to navigate the lake `episodes` number of times
    for _ in range(episodes):
        # We reset the environment so we're back at the start, and store the current
        # state in `observation`
        observation = env.reset()
        episode = []
        steps = 0
        while True:
            # We use our policy to determine the best action based on the current state
            action = policy[observation]
            # We tell the agent to take the action and we retrieve the new state,
            # the reward for moving to that state, and also if we are done or not
            observation, reward, done, _ = env.step(action)
            # We'll save a string representation of the environment so we can watch
            # it later
            episode.append(env.render(mode='ansi'))
            steps += 1
            # If we finished and reached the goal
            if done and reward == 1:
                steps_list.append(steps)

                # We save this episode if we reached the goal in fewer steps
                if len(best_episode) == 0 \
                    or len(episode) < len(best_episode):
                    best_episode = episode

                break
            # If we finished but fell in a hole
            elif done and reward == 0:
                misses += 1
                break

    print('----------------------------------------------')
    print('You took an average of {:.0f} steps to get the frisbee'.format(
        statistics.mean(steps_list)))
    print('And you fell in the hole {:.2f}% of the time'.format(
        (misses / episodes) * 100))
    print('----------------------------------------------')

    return best_episode


best_episode = get_score(env, policy)

----------------------------------------------
You took an average of 71 steps to get the frisbee
And you fell in the hole 27.80% of the time
----------------------------------------------


In [15]:
from google.colab import output

for e in best_episode:
    output.clear()
    print(e)
    time.sleep(0.7)

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG



**Challenges**:

- What's the best score you can get?
- To test your understanding, update `value_iteration` so that it no longer builds the policy while finding the values, and create a new function `get_policy` that builds the policy based on the values returned from `value_iteration`.
    - Hint: for every state, you will need to find the action which gives you the maximum `calc_action_value`, and save that action.

## Sources

- Based on [this article](https://medium.com/analytics-vidhya/solving-the-frozenlake-environment-from-openai-gym-using-value-iteration-5a078dffe438)